In [23]:
# model = get_unet(img_rows, img_cols)
# model.save('../exp.h5')
model3=load_model('../exp.h5')
model3.summary()


# imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
# np.save(predict_path+'predict_exp_%s.npy'%i, imgs_mask_test)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1024, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 1024, 1024, 3 320         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_127 (BatchN (None, 1024, 1024, 3 128         conv2d_134[0][0]                 
__________________________________________________________________________________________________
activation_127 (Activation)     (None, 1024, 1024, 3 0           batch_normalization_127[0][0]    
____________________________________________________________________________________________

# 예측 데이터(predict)

In [11]:
for i in range(5):
    ## npy file #`#
    npy_path = '../4th/data/%s/aug/npy/'%i
    predict_path = '../4th/result/%s/npy/'%i
        
    imgs_train, imgs_mask_train, imgs_test = load_data(npy_path+'train_1024.npy', npy_path+'label_1024.npy', npy_path+'test_1024.npy')
    imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
    np.save(predict_path+'predict_total_%s.npy'%i, imgs_mask_test)

------------------------------
0 data load...
------------------------------
load train images...
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
------------------------------
load test images...
------------------------------
(1955, 1024, 1024, 1)
(1955, 1024, 1024, 1)
(98, 1024, 1024, 1)
98/98 [==============================] - 10s 103ms/step
------------------------------
1 data load...
------------------------------
load train images...
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
------------------------------
load test images...
------------------------------
(1955, 1024, 1024, 1)
(1955, 1024, 1024, 1)
(98, 1024, 1024, 1)
98/98 [==============================] - 10s 102ms/step
------------------------------
2 data load...
----------------------

# 예측마스크 생성

In [2]:
pwd

'/home/ubuntu/gcubme4/Workspace/HM_LEE/spine_scoliosis/total_code'

In [12]:
import os
for i in range(1):
    pred_img_path = '../7th_total/result/%s/img/mask/'%i

    pred_list=np.load('../7th_total/result/%s/npy/predict_exp_%s.npy'%(i,i))
#     print("array to pred image")
    imgs = pred_list
    for i in range(imgs.shape[0]):

        img = imgs[i]
        img[img <= 0.5] = 0
        img[img > 0.5] = 255
        
        img_resize = cv2.resize(img, (1024,1024)) #resize
        cv2.imwrite(pred_img_path +"pred_%d.png" %(i),img_resize)
#         img = array_to_img(img)

#         plt.imshow(img, cmap='gray')
#         plt.axis('off')
#         plt.show()
#         img.save(pred_img_path +"%d_pred.png" %(i))

# Validation

In [ ]:
pwd

In [ ]:
# 한부분씩 검사
true_list=np.load('../../1st/data/4/aug/npy/test_label_1024_lat.npy') #test data mask
true_list=true_list.astype('float32')
true_list = true_list/255.0
true_list[true_list > 0.5] = 1
true_list[true_list <= 0.5] = 0
print(true_list.shape)

pred_list=np.load('../../1st/result/4/img/mask/crop/lat/lat.npy')
pred_list[pred_list > 0.5] = 1
pred_list[pred_list <= 0.5] = 0
# pred_list[pred_list > 127] = 1
# pred_list[pred_list <= 127] = 0

sensitivity=[]
specificity=[]
acc=[]
dsc=[]

for i in range(len(true_list)):
    yt=true_list[i].flatten()
    yp=pred_list[i].flatten()
    mat=confusion_matrix(yt,yp)
    if len(mat) == 2:
        ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
        st=mat[1,1]/(mat[1,0]+mat[1,1])
        sp=mat[0,0]/(mat[0,1]+mat[0,0])
        if mat[1,0]+mat[1,1] == 0:
            specificity.append(sp)
            acc.append(ac)
        else:
            sensitivity.append(st)  
            specificity.append(sp)
            acc.append(ac)
    else:
        specificity.append(1)
        acc.append(1)

for i in range(len(true_list)):
    yt=true_list[i]
    yp=pred_list[i]
    if np.sum(yt) != 0 and np.sum(yp) != 0:
        dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
        dsc.append(dice)

        
print("complete")      
print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
std_sensitivity = np.std(sensitivity)
print("sensitivity std : {0:0.4f}".format(np.mean(std_sensitivity)))
print("")

print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
std_dsc = np.std(dsc)
print("dsc std : {0:0.4f}".format(np.mean(std_dsc)))
print("")

# print("recall = ",recall)
# print("f1score = ",f1score)
# print("accuracy = ",acc)
# print("auc = ",auc)
# print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
# std_specificity = np.std(specificity)
# print("acc avg : {0:0.4f}".format(np.mean(acc)))

# print("sensitivity min:",np.min(sensitivity))
# print("specificity min:",np.min(specificity))
# print("dsc min:",np.min(dsc))
# print("acc min:",np.min(acc))

# print("sensitivity max:",np.max(sensitivity))
# print("specificity max:",np.max(specificity))
# print("dsc max:",np.max(dsc))
# print("acc max:",np.max(acc))

# 5-folder 한번에

In [18]:
pwd

'/home/ubuntu/gcubme4/Workspace/HM_LEE/spine_scoliosis/total_code'

In [12]:
from sklearn.metrics import precision_score
import numpy as np
from metrics import *
from losses import *
import glob
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
import cv2
from keras.models import load_model
import os
from keras.callbacks import EarlyStopping
from tensorflow.compat.v1.keras import backend as K

In [ ]:
# 윤지 검사 코드 (추가 평가지표 참고용)
import cv2
import numpy as np
import pandas as pd
#from data import *
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score,roc_curve,auc
from keras.preprocessing.image import array_to_img
from keras.models import Model, load_model

cv_cnt=0

# for cv_cnt in range(2,5):
mydata = dataProcess(512, 512)
X_test,y_test=mydata.load_test_data(cv_cnt)

print('-'*30)
print('load model...')
print('-'*30)

model=load_model('../result/model/model_normalize_last(cv%s).h5'%str(cv_cnt))

y_pred = model.predict(X_test, batch_size=1, verbose=1)
#np.save('../result/%s/npy/test_pred_epoch300(cv%s).npy'%(str(cv_cnt), str(cv_cnt)), y_pred)
# y_pred=np.load('../result/npy/test_pred(cv%s).npy'%cv_cnt)
pred_label=[]
sensitivity=[]
specificity=[]

f=open("../result/%s/txt/result(cv%s).txt"%(str(cv_cnt),str(cv_cnt)),"w")
pred_label=[]
for i,v in enumerate(y_pred):
    pre_ans=v.argmax()
    pred_label.append(pre_ans)
    txt= str(name_li[i][:7])+","+str(name_li[i][8:9])+","+str(y_test[i])+","+"N-{0:0.2f}".format(v[0]*100)+":"+"P-{0:0.2f}".format(v[1]*100)+","+str(pre_ans)+"\n"
    f.write(txt)
f.close()


cm=confusion_matrix(y_test,pred_label)
print(cm)
if len(cm) == 2:
    st=cm[1,1]/(cm[1,0]+cm[1,1]) #TP/FN+TP
    sp=cm[0,0]/(cm[0,1]+cm[0,0]) #TN/FP+TN
    npv=cm[0,0]/(cm[0,0]+cm[1,0]) #TN/TN+FN
    if cm[1,0]+cm[1,1] == 0:
        specificity.append(sp)
    else:
        sensitivity.append(st)  
        specificity.append(sp)
        negativePvalue.append(npv)
else:
        specificity.append(1)
        
acc=accuracy_score(y_test,pred_label)
precision=precision_score(y_test,pred_label)
f1score=f1_score(y_test,pred_label)
recall=recall_score(y_test,pred_label)
fp,tp,_=roc_curve(y_test,y_pred[:,1])
auc=auc(fp,tp)

print('done')
print("cv_cnt =======================",cv_cnt)
print("precision = ",precision)
# print("recall = ",recall)
print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
print("negative predictable value avg : {0:0.4f}".format(np.mean(negativePvalue)))
print("f1score = ",f1score)
print("accuracy = ",acc)
print("auc = ",auc)


f=open('../result/cv/result_cv{}.txt'.format(cv_cnt),"w")
f.write('cv{} result\n'.format(cv_cnt))
f.write('=====================\n')
f.write("cm = {} \n".format(cm))
f.write("sensitivity avg : {0:0.4f} \n".format(np.mean(sensitivity)))
f.write("specificity avg : {0:0.4f} \n".format(np.mean(specificity)))
f.write("negative predictable value avg : {0:0.4f} \n".format(np.mean(negativePvalue)))
f.write("precision = {} \n".format(precision))
# f.write("recall = {} \n".format(recall))
f.write("f1score ={} \n".format(f1score))
f.write("accuracy = {} \n".format(acc))
f.write("auc = {} \n".format(auc))
f.close()

In [13]:
precision_all=[]

for i in range(5):
   
    true_list=np.load('../4th/data/%s/aug/npy/test_label_1024.npy'%i) #test data mask
    true_list=true_list.astype('float32')
    true_list = true_list/255.0
    true_list[true_list > 0.5] = 1
    true_list[true_list <= 0.5] = 0
    print("%d번째 "%i,true_list.shape)
    print("")
    pred_list=np.load('../4th/result/%s/npy/predict_total_%d.npy'%(i,i))
    pred_list[pred_list > 0.5] = 1
    pred_list[pred_list <= 0.5] = 0
    # pred_list[pred_list > 127] = 1
    # pred_list[pred_list <= 127] = 0
    for i in range(len(true_list)):
        yt=true_list[i].flatten()
        yp=pred_list[i].flatten()
            
    precision=precision_score(yt,yp)
    print("precision avg : {0:0.4f}".format(np.mean(precision)))
    std_precision = np.std(precision)
    print("precision std : {0:0.4f}".format(np.mean(std_precision)))
    precision_all.append(np.mean(precision))
    print("")
    ###############################################################################
    sensitivity=[]
    specificity=[]
    acc=[]
    dsc=[]

    for i in range(len(true_list)):
        yt=true_list[i].flatten()
        yp=pred_list[i].flatten()
        cm=confusion_matrix(yt,yp)
        if len(mat) == 2:
            ac=(cm[1,1]+cm[0,0])/(cm[1,0]+cm[1,1]+cm[0,1]+cm[0,0])            
            st=cm[1,1]/(cm[1,0]+cm[1,1]) #TP/FN+TP
            sp=cm[0,0]/(cm[0,1]+cm[0,0]) #TN/FP+TN
            npv=cm[0,0]/(cm[0,0]+cm[1,0]) #TN/TN+FN
            
            if mat[1,0]+mat[1,1] == 0:
                specificity.append(sp)
                acc.append(ac)
            else:
                sensitivity.append(st)  
                specificity.append(sp)
                acc.append(ac)
                negativePvalue.append(npv)
        else:
            specificity.append(1)
            acc.append(1)

    for i in range(len(true_list)):
        yt=true_list[i]
        yp=pred_list[i]
        if np.sum(yt) != 0 and np.sum(yp) != 0:
            dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
            dsc.append(dice)
    
    print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
    std_sensitivity = np.std(sensitivity)
    print("sensitivity std : {0:0.4f}".format(np.mean(std_sensitivity)))
    print("")

    print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
    std_dsc = np.std(dsc)
    print("dsc std : {0:0.4f}".format(np.mean(std_dsc)))
    print("")
    
    print("negative predictable value avg : {0:0.4f}".format(np.mean(negativePvalue)))
    print("")
    print("")
    print("")
    
std_precision_all= np.std(precision_all)
print("ALL PRE AVG: {0:0.4f}".format(np.mean(precision_all)))
print("ALL PRE STD: {0:0.4f}".format(np.mean(std_precision_all)))

0번째  (98, 1024, 1024, 1)

precision avg : 0.9347
precision std : 0.0000

sensitivity avg : 0.9525
sensitivity std : 0.0384

dsc avg : 0.9589
dsc std : 0.0251



1번째  (98, 1024, 1024, 1)

precision avg : 0.9371
precision std : 0.0000

sensitivity avg : 0.9617
sensitivity std : 0.0239

dsc avg : 0.9641
dsc std : 0.0165



2번째  (98, 1024, 1024, 1)

precision avg : 0.9823
precision std : 0.0000

sensitivity avg : 0.9533
sensitivity std : 0.0425

dsc avg : 0.9585
dsc std : 0.0281



3번째  (98, 1024, 1024, 1)

precision avg : 0.9769
precision std : 0.0000

sensitivity avg : 0.9630
sensitivity std : 0.0266

dsc avg : 0.9654
dsc std : 0.0188



4번째  (97, 1024, 1024, 1)

precision avg : 0.9333
precision std : 0.0000

sensitivity avg : 0.9553
sensitivity std : 0.0312

dsc avg : 0.9608
dsc std : 0.0210



ALL PRE AVG: 0.9528
ALL PRE STD: 0.0220


# 원본+예측 마스크 비교

In [ ]:
for i in range(5):
    pred_img_path = '../../1st/result/%d/img/mask/'%i
    pred_list=np.load('../../1st/result/%d/npy/ap_exp%s.npy'%(i,i))
                      
    pred_list[pred_list > 0.5] = 1
    pred_list[pred_list <= 0.5] = 0

    if not os.path.isdir(pred_img_path):
        os.makedirs(pred_img_path)
        
    pred_overlay_path = '../../1st/result/%d/img/overlay/'%i
    if not os.path.isdir(pred_overlay_path):
        os.makedirs(pred_overlay_path)

    print("array to overlay image")
    imgs = pred_list
    ori_path = glob.glob('../../1st/data/%d/non_aug/test_ori/' + '/*.png'%i)

    for i in range(imgs.shape[0]):
        img = imgs[i]
        ori_imgs = load_img(ori_path[i])
    #     ori_imgs = ori_imgs.resize((512,512))
        ori_img = img_to_array(ori_imgs)
        img_name = ori_path[i][ori_path[i].rindex('/')+1:ori_path[i].rindex('.')]
        print(img_name)
        img[img <= 0.5] = 0
        img[img > 0.5] = 255
        img = img_to_array(img)
        ori_img[:,:,2] = img[:,:,0]
        pred_img = array_to_img(ori_img)



        plt.imshow(pred_img, cmap='gray')
        plt.axis('off')
        plt.show()
        #pred_img.save(pred_overlay_path+"{}_pred.png".format(img_name))

## result picture (원본 원본마스크 결과마스크 원본+예측마스크)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
bigpath="../4th/"
for i in range(3,4):
    ori=glob.glob(bigpath+"data/%d/non_aug/test_ori"%i + '/*.png') #테스트.png 파일 경로
    target=glob.glob(bigpath+"data/%d/non_aug/test_msk"%i + '/*.png')

    pre=glob.glob(bigpath+"result/%d/img/mask"%i + '/*.png')
    result_save_path=bigpath+"result/%d/img/compare/"%i

    for a in range(len(ori)):

        img_name = ori[a][ori[a].rindex('/')+1:ori[a].rindex('.')]
        img1 = plt.imread(ori[a])
        img2 = plt.imread(target[a])
        img3 = plt.imread(pre[a])

        img_name = ori[a][ori[a].rindex('/')+1:ori[a].rindex('.')]

        img_back = cv2.imread(ori[a])
        img_back2 = cv2.imread(ori[a])

        img_color = cv2.imread(pre[a])

        img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY) #이미지의 색상공간 변경 1 channel

        contours, hierarchy = cv2.findContours(img_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            test = cv2.drawContours(img_back, [cnt], 0, (255,0, 0,0), -1)

        plt.rc('font', size=8)
        plt.figure(figsize=(10,5),dpi=300)   
        plt.subplots_adjust(wspace=0.05)

        plt.subplot(1,4,1)
        
        plt.imshow(img1,cmap='gray')
        plt.title('ori')


        plt.axis('off')

        plt.subplot(1,4,2)
        plt.imshow(img2,cmap='gray')
        plt.title('GT')

        plt.axis('off')

        plt.subplot(1,4,3)
        plt.imshow(img3,cmap='gray')
        plt.title('predict')

        plt.axis('off')

        plt.subplot(1,4,4)
        plt.imshow(test)
        plt.imshow(img_back2,alpha=0.5)
        plt.title('ori+predict')

        plt.axis('off')

        plt.savefig(result_save_path+"{}_result.png".format(img_name))

# Morphology 크기정하기

In [1]:
pred_img_path = "../../1st/result/0/img/mask/"
ori_img_path=glob.glob("../../1st/data/0/non_aug/test_msk/" + '/*.png')
ori_img_path.sort()
for i in range(3):
    img1=plt.imread(ori_img_path[i])
    img=pred_img_path +"%d_pred.png" %(i)
    img = plt.imread(img)
    
    plt.figure(figsize=(20,20))
    
    plt.subplot(1,3,1)
    plt.imshow(img1,cmap='gray')
    plt.title('GT')
    plt.axis('off')
    
    #침식3
    kernel = np.ones((6,6), np.uint8)
    result = cv2.erode(img, kernel, iterations = 1)
    
    #팽창3
    kerne2 = np.ones((6,6), np.uint8)
    result = cv2.dilate(result, kerne2, iterations = 1)

    
    plt.subplot(1,3,2)
    plt.imshow(img,cmap='gray')
    plt.title('MO5')
    plt.axis('off')
    
    #침식4
    kernel = np.ones((7,7), np.uint8)
    result = cv2.erode(img, kernel, iterations = 1)
    
    #팽창4
    kerne2 = np.ones((7,7), np.uint8)
    result = cv2.dilate(result, kerne2, iterations = 1)
    result=np.reshape(result,(1024,1024,1))
    
    plt.subplot(1,3,3)
    plt.imshow(result,cmap='gray')
    plt.title('MO6')
    plt.axis('off')

NameError: name 'glob' is not defined

In [ ]:
for a in range(5):
    
    npy_list=[]
    
    save_img_path = "../../1st/result/%d/morphology/7/"%a
    pred_img_path="../../1st/result/%d/img/mask/"%a
    pred_npy_path="../../1st/result/%d/morphology/7/"%a
    ori_img_path=glob.glob("../../1st/data/%d/non_aug/test_msk/"%a +'/*.png')
    ori_img_path.sort()
    
    for i in range(len(ori_img_path)):
        img_name = ori_img_path[i][ori_img_path[i].rindex('/')+1:ori_img_path[i].rindex('.')]

        img=pred_img_path +"%d_pred.png" %(i)
        img = plt.imread(img)

        #침식3
        kernel = np.ones((7,7), np.uint8)
        result = cv2.erode(img, kernel, iterations = 1)

        #팽창3
        kerne2 = np.ones((7,7), np.uint8)
        result2 = cv2.dilate(result, kerne2, iterations = 1)
        
        
        
        outer_result=np.reshape(result2,(1024,1024,1))
        npy_list.append(outer_result)
        print(outer_result.shape)
        
        outer_result=array_to_img(outer_result)
        
        
        plt.imshow(outer_result, cmap='gray')
        plt.axis('off')
        
#         outer_result.save(save_img_path+"%d_MO6.png"%(i))

    np.save(pred_npy_path+'%d_MO6.npy'%a, npy_list)

# 다이콤 정보 엑셀로

In [8]:
import pydicom

def collect_info(filename):
    ds = pydicom.dcmread(filename)
    Series=[]
#     if ds.SeriesDescription not in Series:
    info = []

    info['PatientName']=ds.PatientName

    info['SeriesDescription']=ds.SeriesDescription
    Series.append(ds.SeriesDescription)
    getRepetitionTime(ds)
    getEchoTime(ds)
    getInversionTime(ds)
    getNumberOfAverages(ds)
    getSpacingBetweenSlices(ds)
    getPercentSampling(ds)
    getPercentPhaseFieldOfView(ds)
    getAcquisitionMatrix(ds)
    getFlipAngle(ds)
    getImagesInAcquisition(ds)
    getPixelSpacing(ds)
    f.write(info['PatientName'])
    f.write("\t")
    f.write(info['SeriesDescription'])
    f.write("\t")
    f.write(info['RepetitionTime'])
    f.write("\t")
    f.write(info['EchoTime'])
    f.write("\t")
    f.write(info['InversionTime'])
    f.write("\t")
    f.write(info['NumberOfAverages'])
    f.write("\t")
    f.write(info['SpacingBetweenSlices'])
    f.write("\t")
    f.write(info['PercentSampling'])
    f.write("\t")
    f.write(info['PercentPhaseFieldOfView'])
    f.write("\t")
    f.write(info['AcquisitionMatrix'])
    f.write("\t")
    f.write(info['FlipAngle'])
    f.write("\t")
    f.write(info['ImagesInAcquisition'])
    f.write("\t")     
    f.write(info['PixelSpacing'])
    f.write("\n")
    f.close()
#     Series.append(ds)

In [38]:
my_List=glob.glob("../TEST/"+'/*.dcm')
for i in range(len(my_List)):
    Metadata = pydicom.dcmread(my_List[i])
    date = Metadata.StudyDate
    name=Metadata.PatientName
    birth=Metadata.PatientBirthDate
    sex=Metadata.PatientSex
    ID=Metadata.PatientID
    age=Metadata.PatientAge
    age=age[:-1]
    age=int(age)
#     Physician=Metadata.RequestingPhysician
#     Institution=Metadata.InstitutionName
#     Address=Metadata.InstitutionAddress
#     PhysicianName=Metadata.ReferringPhysicianName
#     StationName=Metadata.StationName
    print(i,"번 환자")
    print(date)
    print(name)
    print(birth)
    print(ID)
    print(age)

    print("")

0 번 환자
20181101
BAEK JI WON
20050923
36008773
13

1 번 환자
20181127
YEO YEONG SEOK
20030817
27035443
15

2 번 환자
20181208
YOON KEON
19990402
13758133
19

3 번 환자
20180822
KIM KYEONG SOOK
19431015
16340723
74

4 번 환자
20190102
LEE SEONG AE
19630920
18910753
55

5 번 환자
20190110
PARK JOON YONG
19700318
21391713
48

6 번 환자
20181019
KANG SEONG HUI
19301016
22189653
88

7 번 환자
20181211
SA EUN JOO
19810624
24200553
37

8 번 환자
20181214
JUNG SOON YI
19480427
27880963
70

9 번 환자
20181222
MOON HYEONG JOO
19470806
29009953
71

10 번 환자
20180713
JANG SE JA^^^^
19541207
30278963
63

11 번 환자
20181016
MOON IN YEONG
19371202
30318663
80

12 번 환자
20180714
LIM CHOON JA
19420625
33970203
76



KeyboardInterrupt: 

In [16]:
    f = open("../TEST/list", "w")
    ds = pydicom.dcmread(my_List[0])
    Series=[]
#     if ds.SeriesDescription not in Series:
    info = {}

    info['PatientName']=ds.PatientName

    info['SeriesDescription']=ds.SeriesDescription
    Series.append(ds.SeriesDescription)
    getRepetitionTime(ds)
    getEchoTime(ds)
    getInversionTime(ds)
    getNumberOfAverages(ds)
    getSpacingBetweenSlices(ds)
    getPercentSampling(ds)
    getPercentPhaseFieldOfView(ds)
    getAcquisitionMatrix(ds)
    getFlipAngle(ds)
    getImagesInAcquisition(ds)
    getPixelSpacing(ds)
    f.write(info['PatientName'])
    f.write("\t")
    f.write(info['SeriesDescription'])
    f.write("\t")
    f.write(info['RepetitionTime'])
    f.write("\t")
    f.write(info['EchoTime'])
    f.write("\t")
    f.write(info['InversionTime'])
    f.write("\t")
    f.write(info['NumberOfAverages'])
    f.write("\t")
    f.write(info['SpacingBetweenSlices'])
    f.write("\t")
    f.write(info['PercentSampling'])
    f.write("\t")
    f.write(info['PercentPhaseFieldOfView'])
    f.write("\t")
    f.write(info['AcquisitionMatrix'])
    f.write("\t")
    f.write(info['FlipAngle'])
    f.write("\t")
    f.write(info['ImagesInAcquisition'])
    f.write("\t")     
    f.write(info['PixelSpacing'])
    f.write("\n")
    f.close()

TypeError: write() argument must be str, not PersonName